In [1]:
# All imports
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [80]:
# Read all files in this cell
temp_raw = pd.read_csv('../../resources/dataset_weather/us_temperature.csv')
humidity_raw = pd.read_csv('../../resources/dataset_weather/us_humidity.csv')
wind_speed_raw = pd.read_csv('../../resources/dataset_weather/us_wind_speed.csv')
accidents_raw = pd.read_csv('../../resources/us_accidents.csv')

In [84]:
accidents_raw['Start_Time'] = pd.to_datetime(accidents_raw['Start_Time'])
accidents_raw['Date'] = accidents_raw['Start_Time'].dt.date
accidents = accidents_raw.drop(columns=['Start_Time','End_Time', 'Distance(mi)'])
accidents = accidents.sort_values('Date').reset_index(drop=True)

,Severity,City,Date
0,4,Whitehall,2016-01-14
1,4,Whitehall,2016-01-14
2,4,Whitehall,2016-01-14
3,4,Whitehall,2016-01-14
4,4,Whitehall,2016-01-14
...,...,...,...
1128106,3,Los Angeles,2017-12-31
1128107,3,San Bernardino,2017-12-31
1128108,3,Pomona,2017-12-31
1128109,3,Los Angeles,2017-12-31


In [74]:
# Cel to filter dates to a range of 2016 and 2017

# First convert dates to DateTime format
temp_raw['datetime'] = pd.to_datetime(temp_raw['datetime'])
humidity_raw['datetime'] = pd.to_datetime(humidity_raw['datetime'])
wind_speed_raw['datetime'] = pd.to_datetime(wind_speed_raw['datetime'])

# Filter the dates using .loc()
temp = temp_raw.loc[(temp_raw['datetime'] >= '2016-01-01') & (temp_raw['datetime'] < '2018-01-01')]
humidity = humidity_raw.loc[(humidity_raw['datetime'] >= '2016-01-01') & (humidity_raw['datetime'] < '2018-01-01')]
wind_speed = wind_speed_raw.loc[(wind_speed_raw['datetime'] >= '2016-01-01') & (wind_speed_raw['datetime'] < '2018-01-01')]

# Reset indexes
temp = temp.reset_index(drop=True)
humidity = humidity.reset_index(drop=True)
wind_speed = wind_speed.reset_index(drop=True)

/var/folders/5r/4dlwcpf56d76tctrcxgmk99w0000gn/T/ipykernel_74778/3732266852.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  temp_raw['datetime'] = pd.to_datetime(temp_raw['datetime'])
/var/folders/5r/4dlwcpf56d76tctrcxgmk99w0000gn/T/ipykernel_74778/3732266852.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  humidity_raw['datetime'] = pd.to_datetime(humidity_raw['datetime'])
/var/folders/5r/4dlwcpf56d76tctrcxgmk99w0000gn/T/ipykernel_74778/3732266852.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  wind_speed_raw['datetime'] = pd.to_datetime(wind_speed_raw['datetime

In [75]:
# Melting for all the weather dataframes
temp_melted = pd.melt(temp, id_vars=['datetime'], var_name='State', value_name='Temperature')
humidity_melted = pd.melt(humidity, id_vars=['datetime'], var_name='State', value_name='Humidity')
wind_speed_melted = pd.melt(wind_speed, id_vars=['datetime'], var_name='State', value_name='Wind_Speed')

In [76]:
# Converting beaufort windspeed to km/h
beaufort_scale = {
    0: (0, 1),
    1: (1, 5),
    2: (6, 11),
    3: (12, 19),
    4: (20, 28),
    5: (29, 38),
    6: (39, 49),
    7: (50, 61),
    8: (62, 74),
    9: (75, 88),
    10: (89, 102),
    11: (103, 117),
    12: (118, 133),
    13: (134, 149),
    14: (150, 166),
    15: (167, 183),
    16: (184, 201),
    17: (202, 220)
}

# Calculate the mean wind speed for each Beaufort scale number
beaufort_mean_speeds = {b: (rng[0] + rng[1]) / 2 for b, rng in beaufort_scale.items()}

# Map the Beaufort scale values to mean km/h values
wind_speed_melted['Wind_Speed_km'] = wind_speed_melted['Wind_Speed'].map(beaufort_mean_speeds)
wind_speed_melted = wind_speed_melted.drop(columns=['Wind_Speed'])

In [77]:
# Merge all three weather dataframes with inner join
weather = pd.merge(temp_melted, humidity_melted,on=['datetime','State'], how='inner')
weather = pd.merge(weather, wind_speed_melted,on=['datetime','State'], how='inner')

In [78]:
# Convert datetime to date
weather['Date'] = weather['datetime'].dt.date
weather = weather.drop(columns=['datetime'])

# Group on day and state
weather = weather.groupby(['Date', 'State']).mean()

Temperature   Humidity  Wind_Speed_km
Date       State                                               
2016-01-01 Albuquerque     267.674555  67.208333      25.895833
           Atlanta         279.687402  80.166667      26.062500
           Boston          275.848301  63.291667      15.395833
           Charlotte       284.268656  71.958333       8.875000
           Denver          264.457188  62.416667       8.791667
...                               ...        ...            ...
2017-12-11 Pittsburgh      274.298750  52.250000       5.875000
           Portland        283.623333  86.041667      38.562500
           San Diego       287.846667  80.833333       9.104167
           San Francisco          NaN        NaN            NaN
           Seattle         283.255833  88.583333      11.729167

[13300 rows x 3 columns]

In [85]:
# Merge the weather and accident dataframes
weather_accidents = pd.merge(weather, accidents, left_on=['Date', 'State'], right_on=['Date', 'City'], how='inner')

,Date,Temperature,Humidity,Wind_Speed_km,Severity,City
0,2016-02-09,292.042917,67.125000,33.062500,2,Indianapolis
1,2016-02-09,291.355417,68.541667,12.500000,2,Pittsburgh
2,2016-02-09,291.355417,68.541667,12.500000,2,Pittsburgh
3,2016-02-11,292.329167,68.583333,12.270833,4,Pittsburgh
4,2016-02-11,292.329167,68.583333,12.270833,4,Pittsburgh
...,...,...,...,...,...,...
137931,2017-12-11,278.059167,63.416667,7.666667,2,Charlotte
137930,2017-12-11,278.059167,63.416667,7.666667,2,Charlotte
137929,2017-12-11,278.059167,63.416667,7.666667,2,Charlotte
137941,2017-12-11,277.601667,58.250000,5.666667,2,Denver


In [86]:
def norm(data):
    min_val = min(data)
    max_val = max(data)
    normalized_data = ((data - min_val) / (max_val - min_val))
    return normalized_data
def parabool(data):
    min_val = min(data)
    max_val = max(data)
    avg = (min_val + max_val)/2
    out_data = abs(data - avg)
    return norm(out_data)

In [88]:
weather_accidents['Wind_Speed_km_norm'] = norm(weather_accidents['Wind_Speed_km'])
weather_accidents['Temp_norm'] = parabool(weather_accidents['Temperature'])
weather_accidents['Humid_norm'] = norm(weather_accidents['Humidity'])

In [89]:
weather_accidents['weather_score'] = weather_accidents['Wind_Speed_km_norm'] + weather_accidents['Humid_norm'] + weather_accidents['Temp_norm']
display(weather_accidents)

,Date,Temperature,Humidity,Wind_Speed_km,Severity,City,Wind_Speed_km_norm,Temp_norm,Humid_norm,weather_score
0,2016-02-09,292.042917,67.125000,33.062500,2,Indianapolis,0.208699,0.202385,0.644869,1.055953
1,2016-02-09,291.355417,68.541667,12.500000,2,Pittsburgh,0.076910,0.179532,0.660172,0.916614
2,2016-02-09,291.355417,68.541667,12.500000,2,Pittsburgh,0.076910,0.179532,0.660172,0.916614
3,2016-02-11,292.329167,68.583333,12.270833,4,Pittsburgh,0.075441,0.211900,0.660622,0.947964
4,2016-02-11,292.329167,68.583333,12.270833,4,Pittsburgh,0.075441,0.211900,0.660622,0.947964
...,...,...,...,...,...,...,...,...,...,...
138245,2017-12-11,283.255833,88.583333,11.729167,2,Seattle,0.071970,0.089661,0.876672,1.038303
138246,2017-12-11,283.255833,88.583333,11.729167,2,Seattle,0.071970,0.089661,0.876672,1.038303
138247,2017-12-11,283.255833,88.583333,11.729167,2,Seattle,0.071970,0.089661,0.876672,1.038303
138248,2017-12-11,283.255833,88.583333,11.729167,2,Seattle,0.071970,0.089661,0.876672,1.038303


In [91]:
weather_accidents_grouped = weather_accidents.groupby(['Date','City']).mean()
weather_accidents_grouped

Temperature   Humidity  Wind_Speed_km  Severity  \
Date       City                                                             
2016-02-09 Indianapolis    292.042917  67.125000      33.062500  2.000000   
           Pittsburgh      291.355417  68.541667      12.500000  2.000000   
2016-02-11 Pittsburgh      292.329167  68.583333      12.270833  4.000000   
2016-02-13 Pittsburgh      261.966687  67.750000      39.395833  3.000000   
2016-02-14 Pittsburgh      259.403506  69.750000       7.041667  4.000000   
...                               ...        ...            ...       ...   
2017-12-11 Pittsburgh      274.298750  52.250000       5.875000  2.500000   
           Portland        283.623333  86.041667      38.562500  2.307692   
           San Diego       287.846667  80.833333       9.104167  2.652174   
           San Francisco          NaN        NaN            NaN  2.000000   
           Seattle         283.255833  88.583333      11.729167  2.100000   

                          Wind_Speed_km_norm  Temp_norm  Humid_norm  \
Date       City                                                       
2016-02-09 Indianapolis             0.208699   0.202385    0.644869   
           Pittsburgh               0.076910   0.179532    0.660172   
2016-02-11 Pittsburgh               0.075441   0.211900    0.660622   
2016-02-13 Pittsburgh               0.249290   0.797329    0.651620   
2016-02-14 Pittsburgh               0.041927   0.882531    0.673225   
...                                      ...        ...         ...   
2017-12-11 Pittsburgh               0.034449   0.387401    0.484182   
           Portland                 0.243949   0.077445    0.849216   
           San Diego                0.055146   0.062898    0.792953   
           San Francisco                 NaN        NaN         NaN   
           Seattle                  0.071970   0.089661    0.876672   

                          weather_score  
Date       City                          
2016-02-09 Indianapolis        1.055953  
           Pittsburgh          0.916614  
2016-02-11 Pittsburgh          0.947964  
2016-02-13 Pittsburgh          1.698239  
2016-02-14 Pittsburgh          1.597683  
...                                 ...  
2017-12-11 Pittsburgh          0.906033  
           Portland            1.170610  
           San Diego           0.910997  
           San Francisco            NaN  
           Seattle             1.038303  

[8918 rows x 8 columns]

In [ ]:
weather_accidents_grouped.to_csv('weather_accidents.csv',index=False)